In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y1", label="ph")

/tmp/ipykernel_1907/4275043310.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-24 18:57:26.567716: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 18:57:28.831980: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 18:57:28.832096: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 18:57:29.157930: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 18:57:29.882101: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 18:57:29.882921: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[5], [10], [15], [20], [25], [30], [35], [40], [45]],
            regularizers=[0.01, 0.1],
            learning_rate=[0.1])

Testando combinacao1: Hidden Size=[5], regularizer=0.01, learning_rate=0.1
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  1.0828270320000115
Stopped at epoch:  11
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.40992855100000725
Stopped at epoch:  13
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.2871822550000047
Stopped at epoch:  7
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.1994633520000093
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.22459549899997455
Stopped at epoch:  7
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.34433613500004867
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 7 ++++++++++

2024-03-24 19:01:58.651176: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  1.8035918620000757
Stopped at epoch:  0
1/1 [==============================] - 0s 20ms/step
+++++++++++ [3_9] | 4 ++++++++++++++++++
Elapsed time:  0.043771129999981895
Stopped at epoch:  0
1/1 [==============================] - 0s 21ms/step
+++++++++++ [3_9] | 5 ++++++++++++++++++
Elapsed time:  0.029540463999978783
Stopped at epoch:  0
1/1 [==============================] - 0s 21ms/step
+++++++++++ [3_9] | 6 ++++++++++++++++++
Elapsed time:  0.028268265999940922
Stopped at epoch:  0
1/1 [==============================] - 0s 20ms/step
+++++++++++ [3_9] | 7 ++++++++++++++++++
Elapsed time:  0.03634788299996217
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [3_9] | 8 ++++++++++++++++++
Elapsed time:  0.043043901999908485
Stopped at epoch:  0
1/1 [==============================] - 0s 17ms/step
+++++++++++ [3_9] | 9 ++++++++++++++++++
Elapsed time:  0.03244145799999387
Stopped at epoch:  0
1/1 [==============================] - 0s 24ms/

2024-03-24 19:10:39.640242: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  6.517391349000036
Stopped at epoch:  0
1/1 [==============================] - 0s 21ms/step
+++++++++++ [8_1] | 6 ++++++++++++++++++
Elapsed time:  0.03798917300014182
Stopped at epoch:  0
1/1 [==============================] - 0s 17ms/step
+++++++++++ [8_1] | 7 ++++++++++++++++++
Elapsed time:  0.03430720899996231
Stopped at epoch:  0
1/1 [==============================] - 0s 18ms/step
+++++++++++ [8_1] | 8 ++++++++++++++++++
Elapsed time:  0.057146344000102545
Stopped at epoch:  0
1/1 [==============================] - 0s 55ms/step
+++++++++++ [8_1] | 9 ++++++++++++++++++
Elapsed time:  0.0813249119999
Stopped at epoch:  0
1/1 [==============================] - 0s 20ms/step
+++++++++++ [8_1] | 10 ++++++++++++++++++
Elapsed time:  0.03265629899988198
Stopped at epoch:  0
1/1 [==============================] - 0s 17ms/step
+++++++++++ [8_1] | 11 ++++++++++++++++++
Elapsed time:  0.05154430999982651
Stopped at epoch:  0
1/1 [==============================] - 0s 17ms/step
+

...

DataFrame salvo em ../results/metrics_17_6
+++++++++++ [17_7] | 1 ++++++++++++++++++
Elapsed time:  0.270483251999849
Stopped at epoch:  7
1/1 [==============================] - 0s 17ms/step
+++++++++++ [17_7] | 2 ++++++++++++++++++
Elapsed time:  0.231172074999904
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [17_7] | 3 ++++++++++++++++++
Elapsed time:  0.2349157019998529
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [17_7] | 4 ++++++++++++++++++
Elapsed time:  0.24236028300038015
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [17_7] | 5 ++++++++++++++++++
Elapsed time:  0.21506649599996308
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [17_7] | 6 ++++++++++++++++++
Elapsed time:  0.230550102000052
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [17_7] | 7 ++++++++++++++++++
Elapsed time:  0.19926

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_17_8_0,0.808324,0.484517,0.487383,0.488571,0.035858,0.090371,0.119872,0.104254
model_17_8_1,0.831187,0.529428,0.539195,0.536987,0.031581,0.082498,0.107756,0.094384
model_17_8_19,0.835953,0.464564,0.602507,0.541632,0.030689,0.093869,0.092951,0.093437
model_17_8_2,0.835989,0.540214,0.552012,0.548817,0.030683,0.080607,0.104759,0.091973
model_17_8_16,0.836211,0.467926,0.601412,0.542574,0.030641,0.093280,0.093207,0.093245
model_17_8_13,0.836513,0.469039,0.602344,0.543581,0.030585,0.093085,0.092989,0.093040
model_17_8_20,0.836538,0.461855,0.607955,0.543340,0.030580,0.094344,0.091677,0.093089
model_17_8_18,0.836748,0.465752,0.606009,0.544063,0.030541,0.093661,0.092132,0.092942
model_17_8_17,0.836996,0.467825,0.605732,0.544858,0.030494,0.093297,0.092197,0.092780
model_17_8_15,0.837025,0.467838,0.606092,0.545057,0.030489,0.093295,0.092113,0.092739


DataFrame salvo em ../results/metrics_17_8
+++++++++++ [17_9] | 1 ++++++++++++++++++
Elapsed time:  0.21188880599993354
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [17_9] | 2 ++++++++++++++++++
Elapsed time:  0.23455194500002108
Stopped at epoch:  7
1/1 [==============================] - 0s 16ms/step
+++++++++++ [17_9] | 3 ++++++++++++++++++
Elapsed time:  0.20698967900034404
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [17_9] | 4 ++++++++++++++++++
Elapsed time:  0.208017160000054
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [17_9] | 5 ++++++++++++++++++
Elapsed time:  0.22166118100039967
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [17_9] | 6 ++++++++++++++++++
Elapsed time:  0.2901677849999942
Stopped at epoch:  7
1/1 [==============================] - 0s 15ms/step
+++++++++++ [17_9] | 7 ++++++++++++++++++
Elapsed time:  0.3

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_17_9_1,0.821541,0.067520,0.732510,0.636494,0.033386,0.046495,0.064358,0.054901
model_17_9_0,0.845846,0.532297,0.933562,0.868449,0.028839,0.023321,0.015985,0.019868


DataFrame salvo em ../results/metrics_17_9
Testando combinacao18: Hidden Size=[45], regularizer=0.1, learning_rate=0.1
+++++++++++ [18_0] | 1 ++++++++++++++++++
Elapsed time:  0.3192582440001388
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_0] | 2 ++++++++++++++++++
Elapsed time:  0.026288918000318517
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_0] | 3 ++++++++++++++++++
Elapsed time:  0.02692659200010894
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_0] | 4 ++++++++++++++++++
Elapsed time:  0.02690221600005316
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_0] | 5 ++++++++++++++++++
Elapsed time:  0.0262672140001996
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [18_0] | 6 ++++++++++++++++++
Elapsed time:  0.028042206999998598
Stopped at epoch:  0
1/1 [============================